In [25]:
import pandas as pd
import numpy as np

In [4]:
# sets up the API keys
from dotenv import load_dotenv
import os 
load_dotenv(dotenv_path='/Users/inigohohmeyer/keys.env')


True

In [10]:
import requests

HOST = 'https://api.census.gov/data'
year = '2010'  # Specify the desired year
dataset = 'dec/sf1'  # Specify the dataset and sub-dataset
api_key = os.getenv("API_KEY_CENSUS") 

base_url = "/".join([HOST, year, dataset])

predicates = {}

get_vars = ["NAME", "P001001"]

predicates["get"] = ",".join(get_vars) 

predicates["for"] = "state:*"

r = requests.get(base_url, params=predicates)

print(r.text)

[["NAME","P001001","state"],
["Alabama","4779736","01"],
["Alaska","710231","02"],
["Arizona","6392017","04"],
["Arkansas","2915918","05"],
["California","37253956","06"],
["Louisiana","4533372","22"],
["Kentucky","4339367","21"],
["Colorado","5029196","08"],
["Connecticut","3574097","09"],
["Delaware","897934","10"],
["District of Columbia","601723","11"],
["Florida","18801310","12"],
["Georgia","9687653","13"],
["Hawaii","1360301","15"],
["Idaho","1567582","16"],
["Illinois","12830632","17"],
["Indiana","6483802","18"],
["Iowa","3046355","19"],
["Kansas","2853118","20"],
["Maine","1328361","23"],
["Maryland","5773552","24"],
["Massachusetts","6547629","25"],
["Michigan","9883640","26"],
["Minnesota","5303925","27"],
["Mississippi","2967297","28"],
["Missouri","5988927","29"],
["Montana","989415","30"],
["Nebraska","1826341","31"],
["Nevada","2700551","32"],
["New Hampshire","1316470","33"],
["New Jersey","8791894","34"],
["New Mexico","2059179","35"],
["New York","19378102","36"],
["

In [23]:
import requests

def get_tract_data(state_code, api_key):
    base_url = 'https://api.census.gov/data/2020/dec/dp'
    # DP1_0078P = White 
    # DP1_0079P = Black
    # DP1_0080P = American Indian
    # DP1_0081P = Asian
    # DP1_0093P = Hispanic or latino
    get_vars = ["NAME", 'DP1_0078P', 'DP1_0079P', 'DP1_0080P', 'DP1_0081P', 'DP1_0093P']
    query_params = {
        'for': 'tract:*',
        'in': f'state:{state_code}',
        'key': api_key
    }
    query_params['get'] = ",".join(get_vars)

    response = requests.get(base_url, params=query_params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print('Error: Unable to retrieve Census data.')
        return None

In [24]:
state_code = '06'  # Example: California
census_api_key = os.getenv("API_KEY_CENSUS")

data = get_tract_data(state_code, census_api_key)
if data is not None:
    # Process the retrieved data as needed
    # for entry in data[1:]:
    #     name = entry[0]
    #     tract_code = entry[1]
        
    #     print(f"Tract: {name}")
    #     print(f"Tract Code: {tract_code}")
    #     print()
    print(data.text)

[["NAME","DP1_0078P","DP1_0079P","DP1_0080P","DP1_0081P","DP1_0093P","state","county","tract"],
["Census Tract 4001; Alameda County; California","61.6","4.7","0.1","17.7","6.7","06","001","400100"],
["Census Tract 4002; Alameda County; California","71.0","2.0","0.1","10.3","10.3","06","001","400200"],
["Census Tract 4003; Alameda County; California","62.0","10.2","0.5","10.1","9.9","06","001","400300"],
["Census Tract 4004; Alameda County; California","67.2","7.0","0.3","9.7","9.1","06","001","400400"],
["Census Tract 4005; Alameda County; California","53.7","17.2","0.7","8.4","12.0","06","001","400500"],
["Census Tract 4006; Alameda County; California","46.9","24.4","0.6","7.0","10.5","06","001","400600"],
["Census Tract 4007; Alameda County; California","43.2","27.1","0.7","9.3","14.0","06","001","400700"],
["Census Tract 4008; Alameda County; California","41.0","25.4","0.6","14.8","12.8","06","001","400800"],
["Census Tract 4009; Alameda County; California","42.6","25.4","0.5","9.8"

In [39]:
import requests
import pandas as pd

import requests
import pandas as pd

def get_tract_data(api_key):
    base_url = 'https://api.census.gov/data/2020/dec/dp'
    get_vars = ['NAME', 'DP1_0078P', 'DP1_0079P', 'DP1_0080P', 'DP1_0081P', 'DP1_0093P']

    df_list = []

    for state_code in range(1, 57):  # Loop through state codes 1 to 56 (excluding 11, 12, 13, 15, 42, 51)
        query_params = {
            'for': 'tract:*',
            'in': f'state:{state_code:02d}',
            'key': api_key
        }
        query_params['get'] = ",".join(get_vars)

        response = requests.get(base_url, params=query_params)
        if response.status_code == 200:
            data = response.json()
            state_data = data[1:]  # Exclude the header row
            if len(state_data[0]) == len(get_vars) + 3:  # Check if there is an extra column
                state_data = [entry[:9] for entry in state_data]  # Keep only the desired columns
            state_df = pd.DataFrame(state_data, columns=data[0][:9])  # Use first 9 column names
            df_list.append(state_df)
        else:
            print(f'Error: Unable to retrieve Census data for state code {state_code:02d}')

    df = pd.concat(df_list, ignore_index=True)
    return df

In [40]:
dataframe = get_tract_data(os.getenv("API_KEY_CENSUS"))
print(dataframe)

Error: Unable to retrieve Census data for state code 03
Error: Unable to retrieve Census data for state code 07
Error: Unable to retrieve Census data for state code 14
Error: Unable to retrieve Census data for state code 43
Error: Unable to retrieve Census data for state code 52
                                               NAME DP1_0078P DP1_0079P  \
0         Census Tract 201; Autauga County; Alabama      78.3      12.0   
1         Census Tract 202; Autauga County; Alabama      41.0      53.7   
2         Census Tract 203; Autauga County; Alabama      69.8      22.2   
3         Census Tract 204; Autauga County; Alabama      84.3       7.7   
4      Census Tract 205.01; Autauga County; Alabama      75.0      14.6   
...                                             ...       ...       ...   
84409      Census Tract 2; Washakie County; Wyoming      89.6       0.2   
84410   Census Tract 3.01; Washakie County; Wyoming      81.5       0.1   
84411   Census Tract 3.02; Washakie County; W

In [41]:
dataframe

,NAME,DP1_0078P,DP1_0079P,DP1_0080P,DP1_0081P,DP1_0093P,state,county,tract
0,Census Tract 201; Autauga County; Alabama,78.3,12.0,0.3,0.5,4.3,01,001,020100
1,Census Tract 202; Autauga County; Alabama,41.0,53.7,0.1,0.6,2.1,01,001,020200
2,Census Tract 203; Autauga County; Alabama,69.8,22.2,0.4,0.4,3.1,01,001,020300
3,Census Tract 204; Autauga County; Alabama,84.3,7.7,0.4,0.8,3.7,01,001,020400
4,Census Tract 205.01; Autauga County; Alabama,75.0,14.6,0.7,2.2,3.6,01,001,020501
...,...,...,...,...,...,...,...,...,...
84409,Census Tract 2; Washakie County; Wyoming,89.6,0.2,0.9,0.4,9.4,56,043,000200
84410,Census Tract 3.01; Washakie County; Wyoming,81.5,0.1,1.4,0.7,19.8,56,043,000301
84411,Census Tract 3.02; Washakie County; Wyoming,85.2,0.1,0.8,0.3,14.2,56,043,000302
84412,Census Tract 9511; Weston County; Wyoming,90.7,0.6,1.6,0.3,4.9,56,045,951100


In [42]:
dataframe2 = pd.read_csv("tract_jail_rP_gP_pall.csv")

In [43]:
dataframe2

,tract,Name,Incarceration_Rate_rP_gP_pall
0,47037013602,"Nashville, TN",0.2300
1,40109105800,"Central Oklahoma City, Oklahoma City, OK",0.1728
2,39035114700,"Kinsman, Cleveland, OH",0.1704
3,9003501400,"Upper Albany, Hartford, CT",0.1679
4,26163522000,"Core City, Detroit, MI",0.1663
...,...,...,...
73497,47157980200,"Memphis, TN",NaN
73498,26103000500,"Marquette, MI",NaN
73499,48201412100,"South Central Houston, Houston, TX",NaN
73500,72127008500,"San Juan, PR",NaN


In [52]:
dataframe['tract'] = dataframe['tract'].astype(str)
dataframe2['tract'] = dataframe2['tract'].astype(str)

merged_df = dataframe.merge(dataframe2, on='tract', how='inner')

# It seems that the census data does not have tract numbers which match with the numbers from the dataset. 


In [53]:
merged_df

,NAME,DP1_0078P,DP1_0079P,DP1_0080P,DP1_0081P,DP1_0093P,state,county,tract,Name,Incarceration_Rate_rP_gP_pall


In [54]:
acsdata = pd.read_csv("acs2017_census_tract_data.csv")

In [66]:
acsdata['TractId'] = acsdata['TractId'].astype(str)

merged_df = acsdata.merge(dataframe2, left_on='TractId', right_on='tract', how='inner')



In [67]:
merged_df

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment,tract,Name,Incarceration_Rate_rP_gP_pall
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.2,0.0,...,24.5,881,74.2,21.2,4.5,0.0,4.6,1001020100,"Prattville, AL",0.0198
1,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,...,22.2,852,75.9,15.0,9.0,0.0,3.4,1001020200,"Prattville, AL",0.0292
2,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,...,23.1,1482,73.3,21.1,4.8,0.7,4.7,1001020300,"Prattville, AL",0.0086
3,1001020400,Alabama,Autauga County,4267,2001,2266,9.6,80.3,7.1,0.5,...,25.9,1849,75.8,19.7,4.5,0.0,6.1,1001020400,"Prattville, AL",0.0088
4,1001020500,Alabama,Autauga County,9965,5054,4911,0.9,77.5,16.4,0.0,...,21.0,4787,71.4,24.1,4.5,0.0,2.3,1001020500,"Prattville, AL",0.0042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73374,72153750501,Puerto Rico,Yauco Municipio,6011,3035,2976,99.7,0.3,0.0,0.0,...,26.9,1576,59.2,33.8,7.0,0.0,20.8,72153750501,"Yauco, PR",NaN
73375,72153750502,Puerto Rico,Yauco Municipio,2342,959,1383,99.1,0.9,0.0,0.0,...,25.3,666,58.4,35.4,6.2,0.0,26.3,72153750502,"Yauco, PR",NaN
73376,72153750503,Puerto Rico,Yauco Municipio,2218,1001,1217,99.5,0.2,0.0,0.0,...,23.5,560,57.5,34.5,8.0,0.0,23.0,72153750503,"Yauco, PR",NaN
73377,72153750601,Puerto Rico,Yauco Municipio,4380,1964,2416,100.0,0.0,0.0,0.0,...,24.1,1062,67.7,30.4,1.9,0.0,29.5,72153750601,"Yauco, PR",NaN


In [68]:
merged_df = merged_df.dropna(subset=['Incarceration_Rate_rP_gP_pall'])


In [70]:
merged_df.columns

Index(['TractId', 'State', 'County', 'TotalPop', 'Men', 'Women', 'Hispanic',
       'White', 'Black', 'Native', 'Asian', 'Pacific', 'VotingAgeCitizen',
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty',
       'ChildPoverty', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Employed', 'PrivateWork', 'PublicWork',
       'SelfEmployed', 'FamilyWork', 'Unemployment', 'tract', 'Name',
       'Incarceration_Rate_rP_gP_pall'],
      dtype='object')

In [99]:
# Select specific columns from the DataFrame
selected_columns = merged_df[['State', 'Hispanic', 'White', 'Black', 'Native', 'Asian']]

# Print the selected columns
print(selected_columns)


             State  Hispanic  White  Black  Native  Asian
0          Alabama       2.4   86.3    5.2     0.0    1.2
1          Alabama       1.1   41.6   54.5     0.0    1.0
2          Alabama       8.0   61.4   26.5     0.6    0.7
3          Alabama       9.6   80.3    7.1     0.5    0.2
4          Alabama       0.9   77.5   16.4     0.0    3.1
...            ...       ...    ...    ...     ...    ...
72536      Wyoming      17.3   79.4    0.0     0.0    0.0
72537      Wyoming      17.7   78.6    0.0     1.0    0.4
72538      Wyoming       3.0   89.4    1.0     0.1    3.0
72539      Wyoming       0.1   93.6    0.0     0.1    5.4
72798  Puerto Rico      50.0   50.0    0.0     0.0    0.0

[71765 rows x 6 columns]


In [103]:
import pandas as pd
import pandas as pd
import pandas as pd

# Assuming your merged DataFrame is named 'merged_df' with columns 'State', 'Asian', 'White', 'Native', 'Black', 'Hispanic', 'Incarceration_Rate_rP_gP_pall', and 'TotalPop'

# Create a new column for each racial group multiplied by the incarceration rate and TotalPop
merged_df.loc[:, 'Asian_Incarcerated'] = merged_df['Asian'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['TotalPop'] * 0.01
merged_df.loc[:, 'White_Incarcerated'] = merged_df['White'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['TotalPop'] * 0.01
merged_df.loc[:, 'Native_Incarcerated'] = merged_df['Native'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['TotalPop'] * 0.01
merged_df.loc[:, 'Black_Incarcerated'] = merged_df['Black'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['TotalPop'] * 0.01
merged_df.loc[:, 'Hispanic_Incarcerated'] = merged_df['Hispanic'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['TotalPop'] * 0.01



final_df = merged_df.loc[:, ['State', 'TractId' , 'Asian_Incarcerated', 'White_Incarcerated', 'Black_Incarcerated', 'Hispanic_Incarcerated', 'Native_Incarcerated']]



/var/folders/03/t1wbmjmx3w94tnymd1sjbfvm0000gn/T/ipykernel_61962/1622828155.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.loc[:, 'Asian_Incarcerated'] = merged_df['Asian'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['TotalPop'] * 0.01
/var/folders/03/t1wbmjmx3w94tnymd1sjbfvm0000gn/T/ipykernel_61962/1622828155.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df.loc[:, 'White_Incarcerated'] = merged_df['White'] * merged_df['Incarceration_Rate_rP_gP_pall'] * merged_df['To

In [104]:
grouped_df = final_df.groupby('State').sum()


In [105]:
final_df

,State,TractId,Asian_Incarcerated,White_Incarcerated,Black_Incarcerated,Hispanic_Incarcerated,Native_Incarcerated
0,Alabama,1001020100,0.438372,31.526253,1.899612,0.876744,0.000000
1,Alabama,1001020200,0.634224,26.383718,34.565208,0.697646,0.000000
2,Alabama,1001020300,0.203777,17.874154,7.714415,2.328880,0.174666
3,Alabama,1001020400,0.075099,30.152329,2.666022,3.604762,0.187748
4,Alabama,1001020500,1.297443,32.436075,6.863892,0.376677,0.000000
...,...,...,...,...,...,...,...
72536,Wyoming,56043000301,0.000000,21.751789,0.000000,4.739370,0.000000
72537,Wyoming,56043000302,0.044806,8.804379,0.000000,1.982665,0.112015
72538,Wyoming,56045951100,0.419454,12.499729,0.139818,0.419454,0.013982
72539,Wyoming,56045951300,1.800058,31.200998,0.000000,0.033334,0.033334


In [106]:
grouped_df

,Asian_Incarcerated,White_Incarcerated,Black_Incarcerated,Hispanic_Incarcerated,Native_Incarcerated
State,,,,,
Alabama,706.172216,40389.872295,30768.961969,3267.570008,319.257351
Alaska,705.698784,5412.438181,352.953568,694.590189,2163.231683
Arizona,2446.213843,44883.818236,5168.337152,41399.882808,6349.955163
Arkansas,582.708007,26399.518616,12273.887986,3591.098042,217.593529
California,40443.017533,109211.092393,31440.325013,183171.094944,1464.159211
Colorado,2303.728257,48515.746142,5164.038077,25889.582024,563.297742
Connecticut,1661.285726,19550.320753,9999.699060,13117.229588,75.448626
Delaware,516.031693,10063.426611,6251.902516,2031.379873,55.606569
District of Columbia,548.087171,4865.747223,13484.554052,1862.070010,30.356484
